# Module 3: Option Greeks Calculator with Real Market Data

**CRITICAL FOR QUANTITATIVE FINANCE CAREERS**

This notebook implements a production-quality Greeks calculator for option risk management and hedging.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import brentq
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Greeks Calculator Loaded!")

## 1. Black-Scholes Option Pricing

Foundation for Greeks calculation:

$$C = S_0 N(d_1) - Ke^{-rT}N(d_2)$$
$$P = Ke^{-rT}N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

In [ ]:
class OptionsCalculator:
    """Production-quality options pricing and Greeks calculator."""
    
    def __init__(self, S, K, T, r, sigma, option_type='call'):
        """
        Initialize options calculator.
        
        Parameters:
        -----------
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration (years)
        r : float
            Risk-free rate
        sigma : float
            Volatility
        option_type : str
            'call' or 'put'
        """
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type.lower()
        
        # Calculate d1 and d2
        self.d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        self.d2 = self.d1 - sigma*np.sqrt(T)
    
    def price(self):
        """Calculate option price."""
        if self.option_type == 'call':
            return self.S * norm.cdf(self.d1) - self.K * np.exp(-self.r*self.T) * norm.cdf(self.d2)
        else:
            return self.K * np.exp(-self.r*self.T) * norm.cdf(-self.d2) - self.S * norm.cdf(-self.d1)
    
    def delta(self):
        """Calculate Delta: ∂V/∂S (rate of change with respect to stock price)."""
        if self.option_type == 'call':
            return norm.cdf(self.d1)
        else:
            return norm.cdf(self.d1) - 1
    
    def gamma(self):
        """Calculate Gamma: ∂²V/∂S² (rate of change of delta)."""
        return norm.pdf(self.d1) / (self.S * self.sigma * np.sqrt(self.T))
    
    def vega(self):
        """Calculate Vega: ∂V/∂σ (sensitivity to volatility)."""
        return self.S * norm.pdf(self.d1) * np.sqrt(self.T) / 100  # Divide by 100 for 1% change
    
    def theta(self):
        """Calculate Theta: ∂V/∂t (time decay per day)."""
        if self.option_type == 'call':
            theta = ((-self.S * norm.pdf(self.d1) * self.sigma) / (2 * np.sqrt(self.T)) -
                    self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2))
        else:
            theta = ((-self.S * norm.pdf(self.d1) * self.sigma) / (2 * np.sqrt(self.T)) +
                    self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2))
        
        return theta / 365  # Per day
    
    def rho(self):
        """Calculate Rho: ∂V/∂r (sensitivity to interest rate)."""
        if self.option_type == 'call':
            return self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(self.d2) / 100
        else:
            return -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-self.d2) / 100
    
    def all_greeks(self):
        """Return all Greeks as a dictionary."""
        return {
            'Price': self.price(),
            'Delta': self.delta(),
            'Gamma': self.gamma(),
            'Vega': self.vega(),
            'Theta': self.theta(),
            'Rho': self.rho()
        }

# Example calculation
S = 100    # Stock price
K = 100    # Strike (ATM)
T = 0.25   # 3 months
r = 0.05   # 5% risk-free rate
sigma = 0.25  # 25% volatility

call_option = OptionsCalculator(S, K, T, r, sigma, 'call')
put_option = OptionsCalculator(S, K, T, r, sigma, 'put')

print("OPTION PRICING AND GREEKS")
print("="*70)
print(f"Stock Price: ${S:.2f}")
print(f"Strike: ${K:.2f}")
print(f"Time to Expiration: {T*365:.0f} days")
print(f"Risk-free Rate: {r:.2%}")
print(f"Volatility: {sigma:.2%}")
print("="*70)

print("\nCALL OPTION:")
call_greeks = call_option.all_greeks()
for greek, value in call_greeks.items():
    print(f"  {greek:.<15} {value:>12.6f}")

print("\nPUT OPTION:")
put_greeks = put_option.all_greeks()
for greek, value in put_greeks.items():
    print(f"  {greek:.<15} {value:>12.6f}")

# Verify Put-Call Parity
parity_lhs = call_greeks['Price'] - put_greeks['Price']
parity_rhs = S - K * np.exp(-r * T)
print(f"\nPut-Call Parity Check: C - P = {parity_lhs:.6f}, S - Ke^(-rT) = {parity_rhs:.6f}")

## 2. Greeks Visualization

Understanding how Greeks change with market conditions is crucial for risk management.

In [ ]:
# Generate Greeks surface plots
S_range = np.linspace(70, 130, 50)
T_range = np.linspace(0.01, 1, 50)

# Store Greeks
greeks_data = {
    'Delta': np.zeros((len(T_range), len(S_range))),
    'Gamma': np.zeros((len(T_range), len(S_range))),
    'Vega': np.zeros((len(T_range), len(S_range))),
    'Theta': np.zeros((len(T_range), len(S_range)))
}

for i, T_val in enumerate(T_range):
    for j, S_val in enumerate(S_range):
        opt = OptionsCalculator(S_val, K, T_val, r, sigma, 'call')
        greeks_data['Delta'][i, j] = opt.delta()
        greeks_data['Gamma'][i, j] = opt.gamma()
        greeks_data['Vega'][i, j] = opt.vega()
        greeks_data['Theta'][i, j] = opt.theta()

# Create mesh grid
S_mesh, T_mesh = np.meshgrid(S_range, T_range)

# Plot 2D heatmaps
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

greeks_list = ['Delta', 'Gamma', 'Vega', 'Theta']

for idx, greek in enumerate(greeks_list):
    ax = axes[idx // 2, idx % 2]
    
    im = ax.contourf(S_mesh, T_mesh, greeks_data[greek], levels=20, cmap='RdYlGn')
    ax.contour(S_mesh, T_mesh, greeks_data[greek], levels=10, colors='black', alpha=0.3, linewidths=0.5)
    
    # Mark strike price
    ax.axvline(x=K, color='white', linestyle='--', linewidth=2, label=f'Strike = {K}')
    
    ax.set_xlabel('Stock Price ($)', fontsize=12)
    ax.set_ylabel('Time to Expiration (years)', fontsize=12)
    ax.set_title(f'{greek} Surface', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    
    plt.colorbar(im, ax=ax, label=greek)

plt.tight_layout()
plt.show()

## 3. Implied Volatility Calculation

Market's expectation of future volatility, derived from option prices.

In [ ]:
def calculate_implied_volatility(market_price, S, K, T, r, option_type='call', sigma_min=0.01, sigma_max=5.0):
    """
    Calculate implied volatility using Brent's method.
    
    Parameters:
    -----------
    market_price : float
        Observed market price
    S, K, T, r : float
        Option parameters
    option_type : str
        'call' or 'put'
    sigma_min, sigma_max : float
        Search bounds for volatility
    
    Returns:
    --------
    float : Implied volatility
    """
    def objective(sigma):
        """Difference between market price and model price."""
        try:
            opt = OptionsCalculator(S, K, T, r, sigma, option_type)
            return opt.price() - market_price
        except:
            return 1e10
    
    try:
        iv = brentq(objective, sigma_min, sigma_max, maxiter=100)
        return iv
    except:
        return np.nan

# Example: Calculate IV from market price
market_call_price = 6.50  # Hypothetical market price
S_current = 100
K_test = 100
T_test = 0.25
r_test = 0.05

implied_vol = calculate_implied_volatility(market_call_price, S_current, K_test, T_test, r_test, 'call')

print(f"\nIMPLIED VOLATILITY CALCULATION")
print("="*70)
print(f"Market Call Price: ${market_call_price:.2f}")
print(f"Stock Price: ${S_current:.2f}")
print(f"Strike: ${K_test:.2f}")
print(f"Time to Expiration: {T_test*365:.0f} days")
print(f"Risk-free Rate: {r_test:.2%}")
print(f"\nImplied Volatility: {implied_vol:.2%}")

# Verify
opt_check = OptionsCalculator(S_current, K_test, T_test, r_test, implied_vol, 'call')
calculated_price = opt_check.price()
print(f"Calculated Price (using IV): ${calculated_price:.2f}")
print(f"Price Difference: ${abs(calculated_price - market_call_price):.6f}")

## 4. Volatility Smile/Skew with Real Data

Implied volatility varies with strike price, creating a "smile" or "skew".

In [ ]:
# Fetch real option data (example with SPY)
ticker = 'SPY'
stock = yf.Ticker(ticker)

try:
    # Get current price
    current_price = stock.history(period='1d')['Close'].iloc[-1]
    
    # Get options expirations
    expirations = stock.options[:3]  # First 3 expirations
    
    print(f"\n{ticker} Options Analysis")
    print("="*70)
    print(f"Current Stock Price: ${current_price:.2f}")
    print(f"\nAvailable Expirations: {expirations}")
    
    # Get options chain for first expiration
    exp_date = expirations[0]
    opt_chain = stock.option_chain(exp_date)
    
    calls = opt_chain.calls
    puts = opt_chain.puts
    
    # Filter for liquid options
    calls = calls[calls['volume'] > 100].copy()
    puts = puts[puts['volume'] > 100].copy()
    
    print(f"\nAnalyzing expiration: {exp_date}")
    print(f"Number of liquid calls: {len(calls)}")
    print(f"Number of liquid puts: {len(puts)}")
    
    # Calculate time to expiration
    exp_datetime = datetime.strptime(exp_date, '%Y-%m-%d')
    T_exp = (exp_datetime - datetime.now()).days / 365
    
    # Plot implied volatility smile
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Calls
    if len(calls) > 0:
        moneyness_calls = calls['strike'] / current_price
        ax1.scatter(moneyness_calls, calls['impliedVolatility'], alpha=0.6, s=50)
        ax1.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='ATM')
        ax1.set_xlabel('Moneyness (K/S)', fontsize=12)
        ax1.set_ylabel('Implied Volatility', fontsize=12)
        ax1.set_title(f'Call Options - Volatility Smile\n{ticker} {exp_date}', 
                     fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # Puts
    if len(puts) > 0:
        moneyness_puts = puts['strike'] / current_price
        ax2.scatter(moneyness_puts, puts['impliedVolatility'], alpha=0.6, s=50, color='orange')
        ax2.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='ATM')
        ax2.set_xlabel('Moneyness (K/S)', fontsize=12)
        ax2.set_ylabel('Implied Volatility', fontsize=12)
        ax2.set_title(f'Put Options - Volatility Skew\n{ticker} {exp_date}', 
                     fontsize=14, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"\nCould not fetch real options data: {e}")
    print("This is common if market is closed or data is unavailable.")
    print("\nGenerating synthetic volatility smile instead...\n")
    
    # Generate synthetic smile
    strikes = np.linspace(80, 120, 30)
    moneyness = strikes / 100
    
    # Stylized smile: higher IV for OTM puts and calls
    iv_smile = 0.20 + 0.05 * (moneyness - 1)**2 + 0.03 * (1 - moneyness)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(moneyness, iv_smile, 'b-', linewidth=2, label='Implied Volatility')
    ax.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='ATM')
    ax.set_xlabel('Moneyness (K/S)', fontsize=12)
    ax.set_ylabel('Implied Volatility', fontsize=12)
    ax.set_title('Synthetic Volatility Smile', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5. Delta Hedging Strategy

Demonstrate how to hedge an option position using the underlying stock.

In [ ]:
def simulate_delta_hedging(S0, K, T, r, sigma, option_type='call', n_steps=100, n_options=100):
    """
    Simulate delta hedging strategy.
    
    Parameters:
    -----------
    S0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to expiration
    r : float
        Risk-free rate
    sigma : float
        Volatility
    option_type : str
        'call' or 'put'
    n_steps : int
        Number of rehedging steps
    n_options : int
        Number of options sold
    
    Returns:
    --------
    pd.DataFrame : Hedging results
    """
    dt = T / n_steps
    times = np.linspace(0, T, n_steps+1)
    
    # Simulate stock price path
    S = np.zeros(n_steps+1)
    S[0] = S0
    
    for i in range(n_steps):
        dW = np.random.normal(0, np.sqrt(dt))
        S[i+1] = S[i] * np.exp((r - 0.5*sigma**2)*dt + sigma*dW)
    
    # Calculate hedge ratios and portfolio value
    results = []
    
    for i, t in enumerate(times):
        time_remaining = T - t
        
        if time_remaining > 0:
            opt = OptionsCalculator(S[i], K, time_remaining, r, sigma, option_type)
            delta = opt.delta()
            option_value = opt.price()
        else:
            # At expiration
            if option_type == 'call':
                option_value = max(S[i] - K, 0)
                delta = 1.0 if S[i] > K else 0.0
            else:
                option_value = max(K - S[i], 0)
                delta = -1.0 if S[i] < K else 0.0
        
        # Portfolio: Short n_options, long delta*n_options shares
        stock_position = delta * n_options
        portfolio_value = stock_position * S[i] - n_options * option_value
        
        results.append({
            'Time': t,
            'Stock_Price': S[i],
            'Option_Value': option_value,
            'Delta': delta,
            'Stock_Holdings': stock_position,
            'Portfolio_Value': portfolio_value
        })
    
    return pd.DataFrame(results)

# Simulate delta hedging
np.random.seed(42)
hedging_results = simulate_delta_hedging(S0=100, K=100, T=1.0, r=0.05, sigma=0.25, 
                                         option_type='call', n_steps=50, n_options=100)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Stock price path
ax = axes[0, 0]
ax.plot(hedging_results['Time'], hedging_results['Stock_Price'], linewidth=2)
ax.axhline(y=100, color='red', linestyle='--', alpha=0.5, label='Strike')
ax.set_title('Stock Price Path', fontsize=12, fontweight='bold')
ax.set_ylabel('Stock Price ($)', fontsize=10)
ax.legend()
ax.grid(True, alpha=0.3)

# Delta evolution
ax = axes[0, 1]
ax.plot(hedging_results['Time'], hedging_results['Delta'], linewidth=2, color='green')
ax.set_title('Delta (Hedge Ratio)', fontsize=12, fontweight='bold')
ax.set_ylabel('Delta', fontsize=10)
ax.grid(True, alpha=0.3)

# Stock holdings
ax = axes[1, 0]
ax.plot(hedging_results['Time'], hedging_results['Stock_Holdings'], linewidth=2, color='orange')
ax.set_title('Stock Holdings (Shares)', fontsize=12, fontweight='bold')
ax.set_xlabel('Time (years)', fontsize=10)
ax.set_ylabel('Shares', fontsize=10)
ax.grid(True, alpha=0.3)

# Portfolio value
ax = axes[1, 1]
ax.plot(hedging_results['Time'], hedging_results['Portfolio_Value'], linewidth=2, color='purple')
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.set_title('Hedged Portfolio Value', fontsize=12, fontweight='bold')
ax.set_xlabel('Time (years)', fontsize=10)
ax.set_ylabel('Portfolio Value ($)', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDELTA HEDGING SIMULATION RESULTS")
print("="*70)
print(f"Initial Portfolio Value: ${hedging_results['Portfolio_Value'].iloc[0]:.2f}")
print(f"Final Portfolio Value: ${hedging_results['Portfolio_Value'].iloc[-1]:.2f}")
print(f"P&L: ${hedging_results['Portfolio_Value'].iloc[-1] - hedging_results['Portfolio_Value'].iloc[0]:.2f}")
print(f"\nFinal Stock Price: ${hedging_results['Stock_Price'].iloc[-1]:.2f}")
print(f"Final Option Payoff: ${max(hedging_results['Stock_Price'].iloc[-1] - 100, 0):.2f}")

## Key Takeaways

1. **Delta**: Hedge ratio, rate of change of option price with stock price
2. **Gamma**: Convexity, rate of change of delta (highest ATM, near expiration)
3. **Vega**: Sensitivity to volatility (highest ATM, far from expiration)
4. **Theta**: Time decay (negative for long positions)
5. **Rho**: Interest rate sensitivity (less important for short-dated options)
6. **Implied Volatility**: Market's expectation, varies with strike (smile/skew)
7. **Delta Hedging**: Dynamic rebalancing to maintain neutrality

**CRITICAL FOR CAREERS:**
- Greeks knowledge essential for derivatives trading roles
- Risk management requires understanding all Greeks
- Interview questions frequently cover Greeks calculations
- Production systems need fast, accurate Greeks computation